In [2]:
import requests
import json

base_url = "http://localhost:8086"


def check_influxdb_health():
    health_url = f"{base_url}/health"
    response = requests.get(health_url)
    if response.status_code == 200:
        return response.json()
    else:
        return None


def get_database_list():
    db_url = f"{base_url}/query?db=_internal&q=SHOW DATABASES"
    response = requests.get(db_url)
    if response.status_code == 200:
        return response.json()
    else:
        return None


def check_database_exists(db_name):
    db_list = get_database_list()
    if db_list and "results" in db_list and len(db_list["results"]) > 0:
        databases = db_list["results"][0]["series"][0]["values"]
        return any(db[0] == db_name for db in databases)
    return False


def inspect_database(db_name):
    if check_database_exists(db_name):
        db_url = f"{base_url}/query?db={db_name}&q=SHOW MEASUREMENTS"
        response = requests.get(db_url)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    else:
        return f"Database '{db_name}' does not exist."

print(f"Health:{check_influxdb_health()}")
print(f"DBList:{get_database_list()}")
print(f"DBExists: {check_database_exists('weatherflow')}")
print(f"DBInspect: {inspect_database('weatherflow')}")



Health:{'checks': [], 'message': 'ready for queries and writes', 'name': 'influxdb', 'status': 'pass', 'version': '1.8.10'}
DBList:{'results': [{'statement_id': 0, 'series': [{'name': 'databases', 'columns': ['name'], 'values': [['weatherflow'], ['_internal']]}]}]}
DBExists: True
DBInspect: {'results': [{'statement_id': 0, 'series': [{'name': 'measurements', 'columns': ['name'], 'values': [['weatherflow_forecast_current'], ['weatherflow_forecast_daily'], ['weatherflow_forecast_hourly'], ['weatherflow_obs'], ['weatherflow_rapid_wind'], ['weatherflow_stats'], ['weatherflow_system_metrics']]}]}]}


In [24]:
def inspect_series(db_name, series_name):
    if check_database_exists(db_name):
        series_url = f'{base_url}/query?db={db_name}&q=SHOW SERIES FROM "{series_name}"'
        response = requests.get(series_url)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    else:
        return f"Database '{db_name}' does not exist."


series = [
            ["weatherflow_evt_strike"],
            # ["weatherflow_forecast_current"],
            # ["weatherflow_forecast_daily"],
            # ["weatherflow_forecast_hourly"],
            ["weatherflow_obs"],
            ["weatherflow_rapid_wind"],
            ["weatherflow_stats"],
            ["weatherflow_system_metrics"],
        ]

for s in series:
    print(f"SeriesInspect: {inspect_series('weatherflow', s[0])}")
    


SeriesInspect: {'results': [{'statement_id': 0, 'series': [{'columns': ['key'], 'values': [['weatherflow_evt_strike,collector_type=collector_websocket,device_id=407976,device_name=ST-00173404,device_type=ST,serial_number=ST-00173404,station_elevation=22.802942276000977,station_latitude=28.13533,station_longitude=-82.48492,station_name=Crystal\\ Lake\\ Rd,station_time_zone=America/New_York'], ['weatherflow_evt_strike,collector_type=primary,device_id=407976,device_name=ST-00173404,device_type=ST,serial_number=ST-00173404,station_elevation=22.802942276000977,station_latitude=28.13533,station_longitude=-82.48492,station_name=Crystal\\ Lake\\ Rd,station_time_zone=America/New_York']]}]}]}
SeriesInspect: {'results': [{'statement_id': 0, 'series': [{'columns': ['key'], 'values': [['weatherflow_obs,collector_type=collector_rest,device_id=407976,device_name=ST-00173404,device_type=ST,serial_number=ST-00173404,station_elevation=22.802942276000977,station_id=171878,station_latitude=28.13533,statio

In [26]:

def get_series_data(db_name, series_name, start_time, end_time):
    if check_database_exists(db_name):
        query = f'SELECT * FROM "{series_name}" WHERE time >= \'{start_time}\' AND time <= \'{end_time}\''
        data_url = f"{base_url}/query?db={db_name}&q={query}"
        response = requests.get(data_url)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    else:
        return f"Database '{db_name}' does not exist."
    
start_time = "2023-10-01T00:00:00Z"
end_time = "2025-10-02T00:00:00Z"
series_name = "weatherflow_obs"
# print(f"SeriesData: {get_series_data('weatherflow', series_name, start_time, end_time)}")
import pandas as pd
def convert_to_dataframe(series_data):
    if series_data and "results" in series_data and len(series_data["results"]) > 0:
        series = series_data["results"][0]["series"]
        if series:
            columns = series[0]["columns"]
            values = series[0]["values"]
            return pd.DataFrame(values, columns=columns)
    return None
df = convert_to_dataframe(get_series_data('weatherflow', series_name, start_time, end_time))
df



,time,air_density,air_temperature,battery,calculated_absolute_humidity,calculated_beaufort_scale_rating,calculated_dew_point,calculated_dew_point_buck,calculated_heat_index,calculated_sea_level_pressure,...,type,uv,wet_bulb_globe_temperature,wet_bulb_temperature,wind_avg,wind_chill,wind_direction,wind_gust,wind_lull,wind_sample_interval
0,2025-07-14T15:59:44Z,1.15090,33.2,2.77,26.750013,1,27.688359,27.506113,45.480488,1014.676361,...,obs_st,10.00,35.2,29.1,0.5,33.2,295,1.1,0.0,3
1,2025-07-14T15:59:44Z,1.15090,33.2,2.77,26.750013,1,27.688359,27.506113,45.480488,1014.676361,...,obs_st,10.00,35.2,29.1,0.5,33.2,295,1.1,0.0,3
2,2025-07-14T16:00:44Z,1.15090,33.2,2.76,26.750013,1,27.688359,27.506113,45.480488,1014.676361,...,obs_st,10.36,33.5,29.1,1.1,33.2,287,1.6,0.8,3
3,2025-07-14T16:00:44Z,1.15090,33.2,2.76,26.750013,1,27.688359,27.506113,45.480488,1014.676361,...,obs_st,10.36,33.5,29.1,1.1,33.2,287,1.6,0.8,3
4,2025-07-14T16:01:44Z,1.15041,33.3,2.77,26.891528,1,27.784330,27.600674,45.826476,1014.575265,...,obs_st,8.33,33.5,29.1,0.8,33.3,291,1.3,0.3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,2025-07-14T19:56:44Z,1.15214,32.9,2.77,26.329280,0,27.400436,27.222385,44.458131,1014.779144,...,obs_st,8.85,34.6,28.7,0.4,32.9,285,1.0,0.0,3
405,2025-07-14T19:57:44Z,1.15176,33.0,2.76,26.831479,0,27.729712,27.546373,45.252950,1014.778300,...,obs_st,3.13,31.8,29.0,0.3,33.0,221,0.9,0.0,3
406,2025-07-14T19:57:44Z,1.15176,33.0,2.76,26.831479,0,27.729712,27.546373,45.252950,1014.778300,...,obs_st,3.13,31.8,29.0,0.3,33.0,221,0.9,0.0,3
407,2025-07-14T19:58:44Z,1.15127,33.1,2.76,26.973644,0,27.825855,27.641102,45.600928,1014.677203,...,obs_st,4.32,32.6,29.1,0.2,33.1,230,0.7,0.0,3


In [25]:

def check_for_null_entries(df):
    if df is not None:
        null_entries = df[df.isnull().any(axis=1)]
        if not null_entries.empty:
            return null_entries
        else:
            return "No null entries found."
    else:
        return "DataFrame is empty or invalid."

for s in series:
    print(f"NullEntries {s}: {check_for_null_entries(convert_to_dataframe(get_series_data('weatherflow', s[0], start_time, end_time)))}")

NullEntries ['weatherflow_evt_strike']: No null entries found.
NullEntries ['weatherflow_obs']:                      time  air_density  air_temperature  battery  \
0    2025-07-14T15:59:44Z      1.15090             33.2     2.77   
1    2025-07-14T15:59:44Z      1.15090             33.2     2.77   
2    2025-07-14T16:00:44Z      1.15090             33.2     2.76   
3    2025-07-14T16:00:44Z      1.15090             33.2     2.76   
4    2025-07-14T16:01:44Z      1.15041             33.3     2.77   
..                    ...          ...              ...      ...   
404  2025-07-14T19:56:44Z      1.15214             32.9     2.77   
405  2025-07-14T19:57:44Z      1.15176             33.0     2.76   
406  2025-07-14T19:57:44Z      1.15176             33.0     2.76   
407  2025-07-14T19:58:44Z      1.15127             33.1     2.76   
408  2025-07-14T19:58:44Z      1.15127             33.1     2.76   

     calculated_absolute_humidity  calculated_beaufort_scale_rating  \
0               

In [19]:

def delete_missing_series_entries(db_name, series_name):
    if check_database_exists(db_name):
        query = f'DELETE FROM "{series_name}" WHERE collector_type = \'collector_rest\''
        delete_url = f"{base_url}/query?db={db_name}&q={query}"
        response = requests.post(delete_url)
        if response.status_code == 204:
            return f"Entries in series '{series_name}' deleted successfully."
        else:
            return f"Failed to delete entries: {response.text}"
    else:
        return f"Database '{db_name}' does not exist."
    
delete_missing_series_entries('weatherflow', 'weatherflow_obs')

'Failed to delete entries: {"results":[{"statement_id":0}]}\n'

In [ ]:
def test_query(db_name):
    query_url = f'query?db={db_name}&epoch=ms&q=SELECT+mean%28%22wind_speed%22%29+%2A+2.236936+FROM+%22weatherflow_rapid_wind%22+WHERE+%28%22station_name%22+%3D~+%2F%5ECrystal+Lake+Rd%24%2F+AND+%22collector_type%22+%3D~+%2F%5Elocal-udp%24%2F%29+AND+time+%3E%3D+1752687414409ms+and+time+%3C%3D+1752687474409ms+GROUP+BY+time%285s%29+fill%28null%29+ORDER+BY+time+ASC HTTP/1.1" 200 57 "-" "Grafana/12.0.2" 9aabfb6a-626b-11f0-b2ce-e6c251d00ca3 804 2025-07-16 13:37:54{"lvl":"info","ts":"2025-07-16T17:37:54.450246Z","msg":"Executing query","log_id":"0xly7Iy0000","service":"query","query":"SELECT mean(precip_accumulated) * 0.039 FROM weatherflow.autogen.weatherflow_obs WHERE (station_name =~ /^Crystal Lake Rd$/ AND collector_type =~ /^local-udp$/) AND time >= 486844h AND time <= 1752687474408ms GROUP BY time(1m) ORDER BY time ASC"}'
    response = requests.get(f'{base_url}/{query_url}')
    print(response.json())

test_query('weatherflow')

{'error': 'error parsing query: found primary, expected regex at line 1, char 92'}
